In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
from backtesting import Backtest, Strategy
import yfinance as yf

# Initialize and connect to MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Fetch historical data from MT5
def fetch_data(symbol, timeframe, start, end):
    rates = mt5.copy_rates_range(symbol, timeframe, start, end)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df



In [ ]:
# Compute ZigZag indicator
def zigzag(df, depth, deviation, backstep):
    zigzag_points = []
    last_high = last_low = None
    
    for i in range(depth, len(df) - depth):
        is_high = df['high'][i] == df['high'][i - depth:i + depth + 1].max()
        is_low = df['low'][i] == df['low'][i - depth:i + depth + 1].min()
        
        if is_high and (last_high is None or (i - last_high[0] > backstep)):
            zigzag_points.append((i, df['high'][i], 'high'))
            last_high = (i, df['high'][i])
            print(f"High point detected at index {i}: {df['high'][i]}")
        
        if is_low and (last_low is None or (i - last_low[0] > backstep)):
            zigzag_points.append((i, df['low'][i], 'low'))
            last_low = (i, df['low'][i])
            print(f"Low point detected at index {i}: {df['low'][i]}")
            
    return zigzag_points


In [ ]:
# Identify support and resistance levels
def identify_levels(zigzag_points, pip_tolerance=10):
    supports = []
    resistances = []

    highs = [point for point in zigzag_points if point[2] == 'high']
    lows = [point for point in zigzag_points if point[2] == 'low']

    for high in highs:
        close_points = [h[1] for h in highs if abs(h[1] - high[1]) <= pip_tolerance]
        if len(close_points) >= 3:
            support_level = np.mean(close_points)
            supports.append(support_level)
            print(f"Support level detected: {support_level}")

    for low in lows:
        close_points = [l[1] for l in lows if abs(l[1] - low[1]) <= pip_tolerance]
        if len(close_points) >= 3:
            resistance_level = np.mean(close_points)
            resistances.append(resistance_level)
            print(f"Resistance level detected: {resistance_level}")

    return supports, resistances


In [ ]:
# Trading logic
def trade(symbol, supports, resistances):
    price = mt5.symbol_info_tick(symbol).bid
    
    if price in supports:
        print(f"Buying at support level: {price}")
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.1,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": price - 100 * 0.0001,
            "tp": price + 200 * 0.0001,
            "deviation": 10,
            "magic": 234000,
            "comment": "Buy at support",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        print(result)
    
    elif price in resistances:
        print(f"Selling at resistance level: {price}")
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.1,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": price + 100 * 0.0001,
            "tp": price - 200 * 0.0001,
            "deviation": 10,
            "magic": 234000,
            "comment": "Sell at resistance",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        print(result)


In [ ]:
# Backtesting with Backtesting library
class ZigZagStrategy(Strategy):
    def init(self):
        self.zigzag_points = zigzag(self.data.df, depth=12, deviation=5, backstep=3)
        self.supports, self.resistances = identify_levels(self.zigzag_points)
        print(f"Identified support levels: {self.supports}")
        print(f"Identified resistance levels: {self.resistances}")

    def next(self):
        price = self.data.Close[-1]
        print(f"Current price: {price}")

        if price in self.supports:
            print("Price hit support level. Buying.")
            self.buy()
        elif price in self.resistances:
            print("Price hit resistance level. Selling.")
            self.sell()


In [ ]:
# Fetch historical data for backtesting
ticker = "GBPUSD=X"
data = yf.download(ticker, start="2020-01-01", end="2024-06-06")

bt = Backtest(data, ZigZagStrategy, cash=10000, commission=.002)
stats = bt.run()
print(stats)
bt.plot()



In [ ]:
# Main loop
symbols = ["EURUSD", "GBPUSD", "USDJPY", "XAUUSD"]

while True:
    for symbol in symbols:
        df = fetch_data(symbol, mt5.TIMEFRAME_H1, datetime.now() - timedelta(days=10), datetime.now())
        zigzag_points = zigzag(df, depth=12, deviation=5, backstep=3)
        supports, resistances = identify_levels(zigzag_points)
        trade(symbol, supports, resistances)
    time.sleep(60)  # Wait a minute before next check

# Shutdown MT5
mt5.shutdown()

In [ ]:
# Separating the codes

In [ ]:
## Backtesting Script

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from backtesting import Backtest, Strategy

# Compute ZigZag indicator
def zigzag(df, depth, deviation, backstep):
    zigzag_points = []
    last_high = last_low = None

    for i in range(depth, len(df) - depth):
        is_high = df['High'][i] == df['High'][i - depth:i + depth + 1].max()
        is_low = df['Low'][i] == df['Low'][i - depth:i + depth + 1].min()

        if is_high and (last_high is None or (i - last_high[0] > backstep)):
            zigzag_points.append((i, df['High'][i], 'high'))
            last_high = (i, df['High'][i])

        if is_low and (last_low is None or (i - last_low[0] > backstep)):
            zigzag_points.append((i, df['Low'][i], 'low'))
            last_low = (i, df['Low'][i])

    return zigzag_points

# Identify support and resistance levels
def identify_levels(zigzag_points, pip_tolerance=0.003):
    supports = []
    resistances = []

    highs = [point for point in zigzag_points if point[2] == 'high']
    lows = [point for point in zigzag_points if point[2] == 'low']

    def find_zones(points, tolerance, min_points):
        zones = []
        i = 0
        while i < len(points):
            j = i + 1
            while j < len(points) and abs(points[j][1] - points[i][1]) <= tolerance:
                j += 1
            if j - i >= min_points:
                zone = [p[1] for p in points[i:j]]
                zones.append(np.mean(zone))
            i = j
        return zones

    supports = find_zones(lows, pip_tolerance, min_points=3)
    resistances = find_zones(highs, pip_tolerance, min_points=4)

    return supports, resistances

# Backtesting with Backtesting library
class ZigZagStrategy(Strategy):
    depth = 12
    deviation = 5
    backstep = 3
    pip_tolerance = 0.0005
    sl_pips = 40 * 0.0001
    tp_pips = 45 * 0.0001
    
    def init(self):
        self.zigzag_points = zigzag(self.data.df, self.depth, self.deviation, self.backstep)
        self.supports, self.resistances = identify_levels(self.zigzag_points, self.pip_tolerance)
        print(f"Identified support levels: {self.supports}")
        print(f"Identified resistance levels: {self.resistances}")

    def next(self):
        price = self.data.Close[-1]

        # Check if the price is within a small range (tolerance) of support/resistance levels
        if self.supports:
            for support in self.supports:
                if abs(price - support) < self.pip_tolerance:
                    print("Price near support level. Buying.")
                    self.buy(sl=price - self.sl_pips, tp=price + self.tp_pips, size=self.dynamic_lotsize())

        if self.resistances:
            for resistance in self.resistances:
                if abs(price - resistance) < self.pip_tolerance:
                    print("Price near resistance level. Selling.")
                    self.sell(sl=price + self.sl_pips, tp=price - self.tp_pips, size=self.dynamic_lotsize())

    def dynamic_lotsize(self):
        # Calculate dynamic lot size based on account equity and risk management rules
        return 0.1  # Example lot size, replace with your logic

def fetch_and_prepare_data(ticker, start, end, interval):
    data = yf.download(ticker, start=start, end=end, interval=interval)
    data.index = pd.to_datetime(data.index)
    return data

# Fetch historical data for backtesting
ticker = "EURUSD=X"
timeframe = "30m"  # Dynamic timeframe
data = fetch_and_prepare_data(ticker, start="2024-05-01", end="2024-06-06", interval=timeframe)

# Convert index to datetime if needed
data.index = pd.to_datetime(data.index)

# Ensure columns are correctly named for the backtesting library
data.columns = [col.capitalize() for col in data.columns]

# Define a wrapper function to run the backtest with specified parameters
def run_backtest(data, **kwargs):
    for key, value in kwargs.items():
        setattr(ZigZagStrategy, key, value)
        
    bt = Backtest(data, ZigZagStrategy, cash=10000, commission=.002, exclusive_orders=True)
    stats = bt.run()
    return stats, bt

# Run the backtest with the dynamic parameters
stats, bt = run_backtest(data, depth=12, deviation=5, backstep=3, pip_tolerance=0.0005, sl_pips=40 * 0.0001, tp_pips=45 * 0.0001)
print(stats)
bt.plot()


[*********************100%%**********************]  1 of 1 completed


Identified support levels: [1.0759051442146301]
Identified resistance levels: [1.0889687538146973]
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near support level. Buying.
Price near resistance level. Selling.
Price near resistance level. Selling.
Price near resistance level. Selling.
Price

Row(id='2788', ...)

In [ ]:
#live trading mt5

In [2]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# Initialize and connect to MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Fetch recent historical data from MT5
def fetch_recent_data(symbol, timeframe, n_bars):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n_bars)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df


# List of symbols
symbols = ["EURUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "XAUUSD"]

# Define the lot size and pip parameters
initial_lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}
pip_tolerance = 3 * pip_dict # 3 pips





# Compute ZigZag indicator
def zigzag(df, depth, deviation, backstep):
    zigzag_points = []
    last_high = last_low = None

    for i in range(depth, len(df) - depth):
        is_high = df['high'].iloc[i] == df['high'].iloc[i - depth:i + depth + 1].max()
        is_low = df['low'].iloc[i] == df['low'].iloc[i - depth:i + depth + 1].min()

        if is_high and (last_high is None or (i - last_high[0] > backstep)):
            zigzag_points.append((i, df['high'].iloc[i], 'high'))
            last_high = (i, df['high'].iloc[i])

        if is_low and (last_low is None or (i - last_low[0] > backstep)):
            zigzag_points.append((i, df['low'].iloc[i], 'low'))
            last_low = (i, df['low'].iloc[i])

    return zigzag_points

# Identify support and resistance levels
def identify_levels(zigzag_points, pip_tolerance):
    supports = []
    resistances = []

    highs = [point for point in zigzag_points if point[2] == 'high']
    lows = [point for point in zigzag_points if point[2] == 'low']

    def find_zones(points, tolerance, min_points):
        zones = []
        i = 0
        while i < len(points):
            j = i + 1
            while j < len(points) and abs(points[j][1] - points[i][1]) <= tolerance:
                j += 1
            if j - i >= min_points:
                zone = [p[1] for p in points[i:j]]
                zones.append(np.mean(zone))
            i = j
        return zones

    supports = find_zones(lows, pip_tolerance, min_points=3)
    resistances = find_zones(highs, pip_tolerance, min_points=3)

    return supports, resistances

# Execute trades based on strategy
executed_trades = {"buy": set(), "sell": set()}

def execute_trades(symbol, supports, resistances, pip_tolerance, sl_pips, tp_pips):
    price = mt5.symbol_info_tick(symbol).last

    # Check if the price is within a small range (tolerance) of support/resistance levels
    for support in supports:
        if abs(price - support) < pip_tolerance and support not in executed_trades["buy"]:
            print(f"Price {price} near support level {support}. Buying.")
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": 0.1,
                "type": mt5.ORDER_TYPE_BUY,
                "price": price,
                "sl": price - sl_pips,
                "tp": price + tp_pips,
                "deviation": 10,
                "magic": 234000,
                "comment": "Support level buy",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(request)
            print(result)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                executed_trades["buy"].add(support)

    for resistance in resistances:
        if abs(price - resistance) < pip_tolerance and resistance not in executed_trades["sell"]:
            print(f"Price {price} near resistance level {resistance}. Selling.")
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": 0.1,
                "type": mt5.ORDER_TYPE_SELL,
                "price": price,
                "sl": price + sl_pips,
                "tp": price - tp_pips,
                "deviation": 10,
                "magic": 234000,
                "comment": "Resistance level sell",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(request)
            print(result)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                executed_trades["sell"].add(resistance)

# Main function to run the strategy
def main():
    symbol = "GBPUSD"
    timeframe = mt5.TIMEFRAME_M30
    n_bars = 500  # Number of bars to fetch for analysis

    data = fetch_recent_data(symbol, timeframe, n_bars)
    zigzag_points = zigzag(data, depth=12, deviation=5, backstep=3)
    supports, resistances = identify_levels(zigzag_points, pip_tolerance=0.0005)

    print(f"Identified support levels: {supports}")
    print(f"Identified resistance levels: {resistances}")

    execute_trades(symbol, supports, resistances, pip_tolerance=0.0005, sl_pips=40 * 0.0001, tp_pips=45 * 0.0001)

if __name__ == "__main__":
    while True:
        main()
        time.sleep(60)  # Wait for 60 seconds before running the next iteration
    mt5.shutdown()


Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333334]
Identified resistance levels: [1.2938]
Identified support levels: [1.2906433333333

KeyboardInterrupt: 

In [ ]:
#Might be the final work

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# Initialize and connect to MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# List of symbols
symbols = ["EURUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "XAUUSD"]

# Define the lot size and pip parameters
initial_lot = 0.1
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}
min_points = 3

# Fetch recent historical data from MT5
def fetch_recent_data(symbol, timeframe, n_bars):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n_bars)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df

# Compute ZigZag indicator
def zigzag(df, depth, deviation, backstep):
    zigzag_points = []
    last_high = last_low = None

    for i in range(depth, len(df) - depth):
        is_high = df['high'].iloc[i] == df['high'].iloc[i - depth:i + depth + 1].max()
        is_low = df['low'].iloc[i] == df['low'].iloc[i - depth:i + depth + 1].min()

        if is_high and (last_high is None or (i - last_high[0] > backstep)):
            zigzag_points.append((i, df['high'].iloc[i], 'high'))
            last_high = (i, df['high'].iloc[i])

        if is_low and (last_low is None or (i - last_low[0] > backstep)):
            zigzag_points.append((i, df['low'].iloc[i], 'low'))
            last_low = (i, df['low'].iloc[i])

    return zigzag_points

# Identify support and resistance levels
def identify_levels(zigzag_points, pip_tolerance):
    supports = []
    resistances = []

    highs = [point for point in zigzag_points if point[2] == 'high']
    lows = [point for point in zigzag_points if point[2] == 'low']

    def find_zones(points, tolerance, min_points):
        zones = []
        i = 0
        while i < len(points):
            j = i + 1
            while j < len(points) and abs(points[j][1] - points[i][1]) <= tolerance:
                j += 1
            if j - i >= min_points:
                zone = [p[1] for p in points[i:j]]
                zones.append(np.mean(zone))
            i = j
        return zones

    supports = find_zones(lows, pip_tolerance, min_points)
    resistances = find_zones(highs, pip_tolerance, min_points)

    return supports, resistances

# Execute trades based on strategy
executed_trades = {"buy": set(), "sell": set()}

def execute_trades(symbol, supports, resistances, pip_tolerance, sl_pips, tp_pips):
    price = mt5.symbol_info_tick(symbol).last

    # Check if the price is within a small range (tolerance) of support levels
    for support in supports:
        if abs(price - support) <= pip_tolerance and support not in executed_trades["buy"]:
            print(f"Price {price} near support level {support}. Buying.")
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": initial_lot,
                "type": mt5.ORDER_TYPE_BUY,
                "price": price,
                "sl": price - sl_pips,
                "tp": price + tp_pips,
                "deviation": 10,
                "magic": 234000,
                "comment": "Support level buy",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(request)
            print(result)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                executed_trades["buy"].add(support)

    # Check if the price is within a small range (tolerance) of resistance levels
    for resistance in resistances:
        if abs(price - resistance) <= pip_tolerance and resistance not in executed_trades["sell"]:
            print(f"Price {price} near resistance level {resistance}. Selling.")
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": initial_lot,
                "type": mt5.ORDER_TYPE_SELL,
                "price": price,
                "sl": price + sl_pips,
                "tp": price - tp_pips,
                "deviation": 10,
                "magic": 234000,
                "comment": "Resistance level sell",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(request)
            print(result)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                executed_trades["sell"].add(resistance)

# Main function to run the strategy
def main():
    timeframe = mt5.TIMEFRAME_M30
    n_bars = 500  # Number of bars to fetch for analysis
    depth = 12
    deviation = 5
    backstep = 3

    for symbol in symbols:
        pip_tolerance = pip_dict[symbol] * 5  # example tolerance
        sl_pips = 40 * pip_dict[symbol]
        tp_pips = 45 * pip_dict[symbol]

        data = fetch_recent_data(symbol, timeframe, n_bars)
        zigzag_points = zigzag(data, depth, deviation, backstep)
        supports, resistances = identify_levels(zigzag_points, pip_tolerance)

        print(f"{symbol}: Identified support levels: {supports}")
        print(f"{symbol}: Identified resistance levels: {resistances}")

        execute_trades(symbol, supports, resistances, pip_tolerance, sl_pips, tp_pips)

if __name__ == "__main__":
    while True:
        main()
        time.sleep(14400)  # Wait for 14400 seconds (4 hours) before running the next iteration
    mt5.shutdown()


In [13]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy

# Compute ZigZag indicator
def zigzag(df, depth, deviation, backstep):
    zigzag_points = []
    last_high = last_low = None

    for i in range(depth, len(df) - depth):
        is_high = df['High'][i] == df['High'][i - depth:i + depth + 1].max()
        is_low = df['Low'][i] == df['Low'][i - depth:i + depth + 1].min()

        if is_high and (last_high is None or (i - last_high[0] > backstep)):
            zigzag_points.append((i, df['High'][i], 'high'))
            last_high = (i, df['High'][i])

        if is_low and (last_low is None or (i - last_low[0] > backstep)):
            zigzag_points.append((i, df['Low'][i], 'low'))
            last_low = (i, df['Low'][i])

    return zigzag_points

# Identify support and resistance levels
def identify_levels(zigzag_points, pip_tolerance=0.003):
    supports = []
    resistances = []

    highs = [point for point in zigzag_points if point[2] == 'high']
    lows = [point for point in zigzag_points if point[2] == 'low']

    def find_zones(points, tolerance, min_points):
        zones = []
        i = 0
        while i < len(points):
            j = i + 1
            while j < len(points) and abs(points[j][1] - points[i][1]) <= tolerance:
                j += 1
            if j - i >= min_points:
                zone = [p[1] for p in points[i:j]]
                zones.append(np.mean(zone))
            i = j
        return zones

    supports = find_zones(lows, pip_tolerance, min_points=3)
    resistances = find_zones(highs, pip_tolerance, min_points=3)

    return supports, resistances

# Backtesting with Backtesting library
class ZigZagStrategy(Strategy):
    depth = 12
    deviation = 5
    backstep = 3
    pip_tolerance = 0.003
    sl_pips = 40 * 0.0001
    tp_pips = 45 * 0.0001
    trades_taken = 0
    
    def init(self):
        self.zigzag_points = zigzag(self.data.df, self.depth, self.deviation, self.backstep)
        self.supports, self.resistances = identify_levels(self.zigzag_points, self.pip_tolerance)
        print(f"Identified support levels: {self.supports}")
        print(f"Identified resistance levels: {self.resistances}")

    def next(self):
        if self.trades_taken == 0:
            price = self.data.Close[-1]

            # Check if the price is within a small range (tolerance) of support/resistance levels
            if self.supports:
                for support in self.supports:
                    if abs(price - support) < self.pip_tolerance:
                        #print("Price near support level. Selling.")
                        self.sell(sl=price + self.sl_pips, tp=price - self.tp_pips, size=self.dynamic_lotsize())
                        self.trades_taken += 1

            if self.resistances:
                for resistance in self.resistances:
                    if abs(price - resistance) < self.pip_tolerance:
                        #print("Price near resistance level. Buying.")
                        self.buy(sl=price - self.sl_pips, tp=price + self.tp_pips, size=self.dynamic_lotsize())
                        self.trades_taken += 1

    def dynamic_lotsize(self):
        # Calculate dynamic lot size based on account equity and risk management rules
        return 0.1  # Example lot size, replace with your logic

    def on_order_closed(self, order):
        self.supports, self.resistances = identify_levels(self.zigzag_points, self.pip_tolerance)
        self.trades_taken = 0

# Load the CSV file with correct headers
file_path = 'GBPUSD_M30_201901020000_202406282330.csv'

# Read the file and manually split the header
with open(file_path, 'r') as file:
    header = file.readline().strip().split('\t')
data = pd.read_csv(file_path, delimiter='\t', skiprows=1, names=header)

# Ensure the data is sorted by date
data['Time'] = pd.to_datetime(data['<DATE>'] + ' ' + data['<TIME>'])
data.set_index('Time', inplace=True)

# Rename columns to match the expected format
data.rename(columns={
    '<OPEN>': 'Open',
    '<HIGH>': 'High',
    '<LOW>': 'Low',
    '<CLOSE>': 'Close',
    '<TICKVOL>': 'TickVol',
    '<VOL>': 'Volume',
    '<SPREAD>': 'Spread'
}, inplace=True)

# Define a wrapper function to run the backtest with specified parameters
def run_backtest(data, **kwargs):
    for key, value in kwargs.items():
        setattr(ZigZagStrategy, key, value)
        
    bt = Backtest(data, ZigZagStrategy, cash=10000, commission=.002, exclusive_orders=True)
    stats = bt.run()
    return stats, bt

# Run the backtest with the dynamic parameters
stats, bt = run_backtest(data, depth=12, deviation=5, backstep=3, pip_tolerance=0.003, sl_pips=40 * 0.0001, tp_pips=45 * 0.0001)
print(stats)
#bt.plot(resample='8H')


Identified support levels: [1.27155, 1.2844066666666667, 1.3135466666666666, 1.30576, 1.3043533333333335, 1.3020933333333333, 1.3162166666666666, 1.3095966666666667, 1.3200775, 1.3155774999999998, 1.3039366666666667, 1.306055, 1.29766, 1.2922399999999998, 1.2887516666666665, 1.3029033333333333, 1.2989385714285715, 1.2702883333333335, 1.2654379999999998, 1.2582666666666666, 1.26804, 1.26708, 1.2665320000000002, 1.2570642857142857, 1.24929, 1.2512733333333335, 1.23919, 1.2132733333333332, 1.209756, 1.2132266666666667, 1.2054259999999999, 1.216076, 1.23231, 1.24404, 1.2275833333333332, 1.2281633333333335, 1.2200925, 1.28005, 1.2836539999999999, 1.2924566666666666, 1.2867875, 1.2819866666666666, 1.29006, 1.28279, 1.289515, 1.3118557142857143, 1.3106700000000002, 1.30596, 1.3092733333333333, 1.30132, 1.2985525, 1.3048666666666666, 1.298055, 1.2888099999999998, 1.2949, 1.2880825, 1.27349, 1.2415975000000001, 1.2301266666666668, 1.240185, 1.2420375, 1.2290566666666667, 1.2226233333333334, 1.2

In [5]:
import pandas as pd

# Load the CSV file
file_path = 'GBPUSD_M30_201901020000_202406282330.csv'
data = pd.read_csv(file_path)

# Print the column names to check for any discrepancies
print(data.columns)


Index(['<DATE>\t<TIME>\t<OPEN>\t<HIGH>\t<LOW>\t<CLOSE>\t<TICKVOL>\t<VOL>\t<SPREAD>'], dtype='object')


In [ ]:
## MT5 Trading Bot Script

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

# Initialize and connect to MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Fetch historical data from MT5
def fetch_data(symbol, timeframe, start, end):
    rates = mt5.copy_rates_range(symbol, timeframe, start, end)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df

# Compute ZigZag indicator
def zigzag(df, depth, deviation, backstep):
    zigzag_points = []
    last_high = last_low = None

    for i in range(depth, len(df) - depth):
        is_high = df['high'][i] == df['high'][i - depth:i + depth + 1].max()
        is_low = df['low'][i] == df['low'][i - depth:i + depth + 1].min()

        if is_high and (last_high is None or (i - last_high[0] > backstep)):
            zigzag_points.append((i, df['high'][i], 'high'))
            last_high = (i, df['high'][i])
            print(f"High point detected at index {i}: {df['high'][i]}")

        if is_low and (last_low is None or (i - last_low[0] > backstep)):
            zigzag_points.append((i, df['low'][i], 'low'))
            last_low = (i, df['low'][i])
            print(f"Low point detected at index {i}: {df['low'][i]}")

    return zigzag_points

# Identify support and resistance levels
def identify_levels(zigzag_points, pip_tolerance=10):
    supports = []
    resistances = []

    highs = [point for point in zigzag_points if point[2] == 'high']
    lows = [point for point in zigzag_points if point[2] == 'low']

    for high in highs:
        close_points = [h[1] for h in highs if abs(h[1] - high[1]) <= pip_tolerance]
        if len(close_points) >= 3:
            support_level = np.mean(close_points)
            supports.append(support_level)
            print(f"Support level detected: {support_level}")

    for low in lows:
        close_points = [l[1] for l in lows if abs(l[1] - low[1]) <= pip_tolerance]
        if len(close_points) >= 3:
            resistance_level = np.mean(close_points)
            resistances.append(resistance_level)
            print(f"Resistance level detected: {resistance_level}")

    return supports, resistances

# Trading logic
def trade(symbol, supports, resistances):
    price = mt5.symbol_info_tick(symbol).bid
    print(f"Current price for {symbol}: {price}")

    pip_value = 0.0001  # Default pip value for most pairs
    if symbol == "USDJPY":
        pip_value = 0.01
    elif symbol == "XAUUSD":
        pip_value = 0.1

    sl_pips = 40 * pip_value
    tp_pips = 45 * pip_value

    if price in supports:
        print(f"Buying at support level: {price}")
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.1,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": price - sl_pips,
            "tp": price + tp_pips,
            "deviation": 10,
            "magic": 234000,
            "comment": "Buy at support",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        print(result)

    elif price in resistances:
        print(f"Selling at resistance level: {price}")
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.1,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": price + sl_pips,
            "tp": price - tp_pips,
            "deviation": 10,
            "magic": 234000,
            "comment": "Sell at resistance",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        print(result)

# Main loop
symbols = ["EURUSD", "GBPUSD", "USDJPY", "XAUUSD"]

while True:
    for symbol in symbols:
        df = fetch_data(symbol, mt5.TIMEFRAME_H1, datetime.now() - timedelta(days=10), datetime.now())
        zigzag_points = zigzag(df, depth=12, deviation=5, backstep=3)
        supports, resistances = identify_levels(zigzag_points)
        trade(symbol, supports, resistances)
    time.sleep(60)  # Wait a minute before next check

# Shutdown MT5
mt5.shutdown()
